# check oof df

In [1]:
import os
import gc
import yaml
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("/kaggle/src/dss_utils")
sys.path.append("/kaggle/src/exp")
sys.path.append("/kaggle/src/model")
sys.path.append("/kaggle/src/data")
from dss_metrics import score
from training_loop import get_valid_values_dict, concat_valid_input_info, get_oof_df, get_key_df
from dss_model import get_model
from dss_dataloader import get_loader


In [2]:
output_dir = "/kaggle/working"
exp_name = "exp017_inputtargettd_flip_epoch10"

folds = [0, 1, 2, 3, 4]
oof_df = pd.DataFrame()
for i in folds:
    print("fold", i)
    # df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"raw_oof_df_fold{i}.parquet"))
    df = pd.read_parquet(os.path.join(output_dir, "_oof", exp_name, f"oof_df_fold{i}.parquet"))
    oof_df = pd.concat([oof_df, df], axis=0)


fold 0
fold 1
fold 2
fold 3
fold 4


In [4]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
train_event_df = train_event_df[train_event_df["step"].notnull()].copy()
train_event_df = train_event_df.reset_index(drop=True)


In [7]:
score_df = pd.read_csv(os.path.join(output_dir, exp_name, "score_df.csv"))


In [8]:
score_df = score_df.sort_values("score", ascending=True).reset_index(drop=True)


In [9]:
series_id = "f56824b503a0"
def display_series_scoredf(orig_df, pred_df, oof_df, train_event_df, series_id):
    input_df = orig_df[orig_df["series_id"] == series_id]
    pred_df = oof_df[oof_df["series_id"] == series_id]
    key_unique = input_df["series_date_key"].unique()
    print(len(input_df)/17280)
    score_list = []
    key_list = []
    for key in key_unique:
        input_key_df = input_df[input_df["series_date_key"] == key]
        start_step = input_key_df["step"].min()
        end_step = input_key_df["step"].max()
        pred_key_df = pred_df[(pred_df["step"] >= start_step) & (pred_df["step"] <= end_step)]
        scoring_event_df = train_event_df[train_event_df["series_id"] == series_id].copy()
        scoring_event_df = scoring_event_df[(scoring_event_df["step"] >= start_step) & (scoring_event_df["step"] <= end_step)]
        sub_df = make_submission_df(pred_key_df, threshold=0.01)
        if len(sub_df["event"].unique()) < 2 or len(scoring_event_df["event"].unique()) < 2:
            score_list.append(0)
            key_list.append(key)
        else:
            score_list.append(score(scoring_event_df, sub_df))
            key_list.append(key)

    df = pd.DataFrame({"key": key_list, "score": score_list}).sort_values("score").reset_index(drop=True)
    display(pd.DataFrame({"key": key_list, "score": score_list}).sort_values("score", ascending=False).reset_index(drop=True))
    key_list = df["key"].values.tolist()
    return key_list


In [10]:
def show_input_target_in_key(orig_df, oof_df, key, series_id):
    input_df = orig_df[orig_df["series_id"] == series_id]
    pred_df = oof_df[oof_df["series_id"] == series_id]
    input_key_df = input_df[input_df["series_date_key"] == key]
    print(input_key_df["series_id"].unique())
    start_step = input_key_df["step"].min()
    end_step = input_key_df["step"].max()
    pred_key_df = pred_df[(pred_df["step"] >= start_step) & (pred_df["step"] <= end_step)]
    scoring_event_df = train_event_df[train_event_df["series_id"] == series_id].copy()
    scoring_event_df = scoring_event_df[(scoring_event_df["step"] >= start_step) & (scoring_event_df["step"] <= end_step)]
    sub_df = make_submission_df(pred_key_df, threshold=0.01)
    print(scoring_event_df["step"].values)

    plt.figure(figsize=(20, 5))
    plt.subplot(5, 1, 1)
    plt.plot(input_key_df["anglez"])
    plt.subplot(5, 1, 2)
    plt.plot(input_key_df["enmo"])
    plt.subplot(5, 1, 3)
    plt.plot(input_key_df["step"], input_key_df["event"])
    # plt.plot(input_key_df["step"]-12*60*6, input_key_df["event"])
    plt.subplot(5, 1, 4)
    plt.plot(pred_key_df["class_pred"])
    plt.subplot(5, 1, 5)
    plt.plot(pred_key_df["step"], pred_key_df["event_pred"])
    plt.plot(input_key_df[input_key_df["second"]==0]["step"], input_key_df[input_key_df["second"]==0]["event_onset"].values, ":")
    plt.plot(input_key_df[input_key_df["second"]==0]["step"], input_key_df[input_key_df["second"]==0]["event_wakeup"].values, ":")
    plt.tight_layout()
    plt.show()


---
check event

In [15]:
score_df.head(10)


,series_id,score,detected_event_num
0,60d31b0bec3b,0.055131,388
1,f56824b503a0,0.082019,138
2,4feda0596965,0.115795,188
3,e4500e7e19e1,0.127249,24
4,854206f602d0,0.305684,53
5,b7fc34995d0f,0.367857,23
6,c535634d7dcd,0.394813,137
7,7476c0bd18d2,0.405163,45
8,db5e0ee1c0ab,0.435557,551
9,60e51cad2ffb,0.448291,50


In [14]:
event_check_df = train_event_df[train_event_df["series_id"]=="60d31b0bec3b"]
display(event_check_df)


,series_id,night,event,step,timestamp
3693,60d31b0bec3b,2,onset,25644.0,2019-01-11T22:37:00-0500
3694,60d31b0bec3b,2,wakeup,30204.0,2019-01-12T04:57:00-0500
3695,60d31b0bec3b,3,onset,42696.0,2019-01-12T22:18:00-0500
3696,60d31b0bec3b,3,wakeup,48408.0,2019-01-13T06:14:00-0500
3697,60d31b0bec3b,5,onset,78816.0,2019-01-15T00:28:00-0500
3698,60d31b0bec3b,5,wakeup,83196.0,2019-01-15T06:33:00-0500
3699,60d31b0bec3b,6,onset,95844.0,2019-01-16T00:07:00-0500
3700,60d31b0bec3b,6,wakeup,100656.0,2019-01-16T06:48:00-0500
3701,60d31b0bec3b,7,onset,111960.0,2019-01-16T22:30:00-0500
3702,60d31b0bec3b,7,wakeup,119016.0,2019-01-17T08:18:00-0500


In [16]:
event_check_df = train_event_df[train_event_df["series_id"]=="f56824b503a0"]
display(event_check_df)


,series_id,night,event,step,timestamp
9170,f56824b503a0,2,onset,24228.0,2018-10-02T21:24:00-0400
9171,f56824b503a0,2,wakeup,30048.0,2018-10-03T05:29:00-0400
9172,f56824b503a0,3,onset,41616.0,2018-10-03T21:33:00-0400
9173,f56824b503a0,3,wakeup,47016.0,2018-10-04T05:03:00-0400
9174,f56824b503a0,4,onset,58584.0,2018-10-04T21:07:00-0400
9175,f56824b503a0,4,wakeup,64476.0,2018-10-05T05:18:00-0400
9176,f56824b503a0,5,onset,76584.0,2018-10-05T22:07:00-0400
9177,f56824b503a0,5,wakeup,81576.0,2018-10-06T05:03:00-0400
9178,f56824b503a0,6,onset,96072.0,2018-10-07T01:11:00-0400
9179,f56824b503a0,6,wakeup,100836.0,2018-10-07T07:48:00-0400
